In [1]:
import json
import os
import _jsonnet
import os

In [2]:
from seq2struct.commands.infer import Inferer
from seq2struct.datasets.spider import SpiderItem
from seq2struct.utils import registry

/opt/conda/envs/gap-text2sql/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

In [4]:
exp_config = json.loads(
    _jsonnet.evaluate_file(
        "experiments/spider-configs/gap-run.jsonnet"))


In [5]:
model_config_path = exp_config["model_config"]
model_config_args = exp_config.get("model_config_args")

In [6]:
infer_config = json.loads(
    _jsonnet.evaluate_file(
        model_config_path, 
        tla_codes={'args': json.dumps(model_config_args)}))

In [7]:
infer_config["model"]["encoder_preproc"]["db_path"] = "data/sqlite_files/"

In [8]:
inferer = Inferer(infer_config)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}


In [9]:
model_dir = exp_config["logdir"] + "/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1"
checkpoint_step = exp_config["eval_steps"][0]

In [10]:
model = inferer.load_model(model_dir, checkpoint_step)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'data/spider-bart/nl2code-1115,output_from=true,fs=2,emb=bart,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'bart_version': 'facebook/bart-large', 'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'data/sqlite_files/', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'save_path': 'data/spider-bart/nl2code-1115,output_from=true,fs=2,emb=bart,cvlink'}}


Parameter containing:
tensor([[-0.0370,  0.1117,  0.1829,  ...,  0.2054,  0.0578, -0.0750],
        [ 0.0055, -0.0049, -0.0069,  ..., -0.0030,  0.0038,  0.0087],
        [-0.0448,  0.4604, -0.0604,  ...,  0.1073,  0.0310,  0.0477],
        ...,
        [-0.0138,  0.0278, -0.0467,  ...,  0.0455, -0.0265,  0.0125],
        [-0.0043,  0.0153, -0.0567,  ...,  0.0496,  0.0108, -0.0099],
        [ 0.0053,  0.0324, -0.0179,  ..., -0.0085,  0.0223, -0.0020]],
       requires_grad=True)
Updated the model with ./pretrained_checkpoint/pytorch_model.bin
Parameter containing:
tensor([[-3.8313e-02,  1.2050e-01,  1.7760e-01,  ...,  1.9729e-01,
          5.9443e-02, -6.9929e-02],
        [ 4.5650e-03, -2.3032e-03, -8.4326e-03,  ..., -3.5686e-03,
          4.7121e-03,  8.4110e-03],
        [-4.5997e-02,  4.6710e-01, -6.5000e-02,  ...,  1.0271e-01,
          2.5631e-02,  4.7501e-02],
        ...,
        [ 1.5723e-02, -2.5663e-02,  5.2457e-03,  ..., -1.6183e-02,
          1.9788e-02, -6.2702e-03],
     

In [11]:
from seq2struct.datasets.spider_lib.preprocess.get_tables import dump_db_json_schema
from seq2struct.datasets.spider import load_tables_from_schema_dict

In [24]:
db_id = "security_1"

In [25]:
my_schema = dump_db_json_schema("data/sqlite_files/{db_id}/{db_id}.sqlite".format(db_id=db_id), db_id)

In [26]:
from seq2struct.utils.api_utils import refine_schema_names

In [27]:
my_schema

{'db_id': 'music_1',
 'table_names_original': ['genre', 'artist', 'files', 'song'],
 'table_names': ['genre', 'artist', 'files', 'song'],
 'column_names_original': [(-1, '*'),
  (0, 'g_name'),
  (0, 'rating'),
  (0, 'most_popular_in'),
  (1, 'artist_name'),
  (1, 'country'),
  (1, 'gender'),
  (1, 'preferred_genre'),
  (2, 'f_id'),
  (2, 'artist_name'),
  (2, 'file_size'),
  (2, 'duration'),
  (2, 'formats'),
  (3, 'song_name'),
  (3, 'artist_name'),
  (3, 'country'),
  (3, 'f_id'),
  (3, 'genre_is'),
  (3, 'rating'),
  (3, 'languages'),
  (3, 'releasedate'),
  (3, 'resolution')],
 'column_names': [(-1, '*'),
  (0, 'g name'),
  (0, 'rating'),
  (0, 'most popular in'),
  (1, 'artist name'),
  (1, 'country'),
  (1, 'gender'),
  (1, 'preferred genre'),
  (2, 'f id'),
  (2, 'artist name'),
  (2, 'file size'),
  (2, 'duration'),
  (2, 'formats'),
  (3, 'song name'),
  (3, 'artist name'),
  (3, 'country'),
  (3, 'f id'),
  (3, 'genre is'),
  (3, 'rating'),
  (3, 'languages'),
  (3, 'released

In [28]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(my_schema)

In [29]:
schema.keys()

dict_keys(['music_1'])

In [30]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": "data/sqlite_files/"
})

In [31]:
for _, schema in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema)

In [32]:
spider_schema = dataset.schemas[db_id]

In [33]:
def infer(question):
    data_item = SpiderItem(
            text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
            code=None,
            schema=spider_schema,
            orig_schema=spider_schema.orig,
            orig={"question": question}
        )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        output = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
    return output[0]["inferred_code"]

In [34]:
code = infer("What type of device operating systems are used by the customer?")
print(code)

/opt/conda/envs/gap-text2sql/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


SELECT Avg(song.rating), Avg(song.resolution) FROM song WHERE song.languages = 'terminal'


In [ ]:
code = infer("Can you provide a breakdown of the different payment types used by customers in the database, and how many customers used each payment type in the last month?")
print(code)

/opt/conda/envs/gap-text2sql/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


SELECT Avg(song.rating), Avg(song.resolution) FROM song WHERE song.languages = 'terminal'


In [ ]:
code = infer("What is the average credit risk score based on age group?")
print(code)

/opt/conda/envs/gap-text2sql/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


SELECT Avg(song.rating), Avg(song.resolution) FROM song WHERE song.languages = 'terminal'
